
Predicting Fake News using NLP and Machine Learning 

Reperformance using Data set with fake news

https://towardsdatascience.com/predicting-fake-news-using-nlp-and-machine-learning-scikit-learn-glove-keras-lstm-7bbd557c3443


## Setup

In [114]:
!pip install contractions

In [115]:
import pandas as pd
import numpy as np



import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import contractions

import string
import joblib
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# set seed
np.random.seed(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## load data

In [116]:
train_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Train.csv', header=0)
val_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Val.csv', header=0)

In [117]:
stopwords = nltk.corpus.stopwords.words('english')

def cleantext(string):
    text = string.lower().split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)    
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)  
    tx = text.replace('&amp',' ')
    text = re.sub(r"[^a-zA-Z]+",' ',text)
    text = text.split()
    text = [w for w in text if not w in stopwords]
    text = " ".join(text)
    return text

In [118]:
train_df['tweet'] = train_df['tweet'].map(lambda x: cleantext(x))
val_df['tweet'] = val_df['tweet'].map(lambda x: cleantext(x))

## Default Version

In [119]:
# Default

count_vectorizer = CountVectorizer(ngram_range=(1, 1))
tf_idf_transformer = TfidfTransformer(smooth_idf=True)

# fit and transform train data to count vectorizer
count_vectorizer.fit(train_df['tweet'].values)
count_vect_train = count_vectorizer.transform(train_df['tweet'].values)

# fit the counts vector to tfidf transformer
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

# Transform the test data as well
count_vect_test = count_vectorizer.transform(val_df['tweet'].values)
tf_idf_test = tf_idf_transformer.transform(count_vect_test)

In [120]:
def get_perf_metrics(model, i):

    # model name
    model_name = type(model).__name__
    # time keeping
    start_time = time.time()
    print("Training {} model...".format(model_name))
    # Fitting of model
    model.fit(tf_idf_train, train_df["label"])
    print("Completed {} model training.".format(model_name))
    elapsed_time = time.time() - start_time
    # Time Elapsed
    print("Time elapsed: {:.2f} s.".format(elapsed_time))
    # Predictions
    y_pred = model.predict(tf_idf_test)
    # Add to ith row of dataframe - metrics
    df_perf_metrics.loc[i] = [
        model_name,
        model.score(tf_idf_train, train_df["label"]),
        model.score(tf_idf_test, val_df["label"]),
        precision_score(val_df["label"], y_pred, pos_label="real"),
        recall_score(val_df["label"], y_pred, pos_label="real"),
        f1_score(val_df["label"], y_pred, pos_label="real"), "{:.2f}".format(elapsed_time)
    ]
    # keep a track of trained models
    models_trained_list.append(model)
    print("Completed {} model's performance assessment.".format(model_name))

In [121]:
models_list = [LogisticRegression(),
               MultinomialNB(),
               RandomForestClassifier(),
               DecisionTreeClassifier(),
               GradientBoostingClassifier(),
               AdaBoostClassifier(),
               LinearSVC()]


df_perf_metrics = pd.DataFrame(columns=['Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision', 'Recall', 'f1_score'])    

df_perf_metrics = pd.DataFrame(columns=[
    'Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision',
    'Recall', 'f1_score', 'Training Time (secs'
])

models_trained_list = []

In [122]:
%%time
for n, model in enumerate(models_list):
    get_perf_metrics(model, n)

df_perf_metrics_default = df_perf_metrics

Training LogisticRegression model...
Completed LogisticRegression model training.
Time elapsed: 0.28 s.
Completed LogisticRegression model's performance assessment.
Training MultinomialNB model...
Completed MultinomialNB model training.
Time elapsed: 0.02 s.
Completed MultinomialNB model's performance assessment.
Training RandomForestClassifier model...
Completed RandomForestClassifier model training.
Time elapsed: 4.48 s.
Completed RandomForestClassifier model's performance assessment.
Training DecisionTreeClassifier model...
Completed DecisionTreeClassifier model training.
Time elapsed: 1.75 s.
Completed DecisionTreeClassifier model's performance assessment.
Training GradientBoostingClassifier model...
Completed GradientBoostingClassifier model training.
Time elapsed: 6.08 s.
Completed GradientBoostingClassifier model's performance assessment.
Training AdaBoostClassifier model...
Completed AdaBoostClassifier model training.
Time elapsed: 2.30 s.
Completed AdaBoostClassifier model's p

### result

ngram_range=(1, 1)
smooth_idf=True

In [123]:
df_perf_metrics_default

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.958100,0.920561,0.932605,0.914286,0.923354,0.28
1,MultinomialNB,0.953427,0.908411,0.885000,0.948214,0.915517,0.02
2,RandomForestClassifier,0.999844,0.904206,0.927171,0.886607,0.906435,4.48
3,DecisionTreeClassifier,1.000000,0.844860,0.842014,0.866071,0.853873,1.75
4,GradientBoostingClassifier,0.893146,0.860748,0.905325,0.819643,0.860356,6.08
5,AdaBoostClassifier,0.867757,0.844393,0.858053,0.841964,0.849932,2.30
6,LinearSVC,0.997664,0.932710,0.938849,0.932143,0.935484,0.07


## Best Parameter Version

In [124]:
# best parameters set according to grid search

count_vectorizer = CountVectorizer(ngram_range=(1, 2))
tf_idf_transformer = TfidfTransformer(smooth_idf=True)

# fit and transform train data to count vectorizer
count_vectorizer.fit(train_df['tweet'].values)
count_vect_train = count_vectorizer.transform(train_df['tweet'].values)

# fit the counts vector to tfidf transformer
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

# Transform the test data as well
count_vect_test = count_vectorizer.transform(val_df['tweet'].values)
tf_idf_test = tf_idf_transformer.transform(count_vect_test)

In [125]:
models_list = [LogisticRegression(),
               MultinomialNB(),
               RandomForestClassifier(),
               DecisionTreeClassifier(),
               GradientBoostingClassifier(),
               AdaBoostClassifier(),
               LinearSVC()]


df_perf_metrics = pd.DataFrame(columns=['Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision', 'Recall', 'f1_score'])    

df_perf_metrics = pd.DataFrame(columns=[
    'Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision',
    'Recall', 'f1_score', 'Training Time (secs'
])

models_trained_list = []

In [126]:
%%time
for n, model in enumerate(models_list):
    get_perf_metrics(model, n)

df_perf_metrics_best = df_perf_metrics


Training LogisticRegression model...
Completed LogisticRegression model training.
Time elapsed: 1.17 s.
Completed LogisticRegression model's performance assessment.
Training MultinomialNB model...
Completed MultinomialNB model training.
Time elapsed: 0.12 s.
Completed MultinomialNB model's performance assessment.
Training RandomForestClassifier model...
Completed RandomForestClassifier model training.
Time elapsed: 11.09 s.
Completed RandomForestClassifier model's performance assessment.
Training DecisionTreeClassifier model...
Completed DecisionTreeClassifier model training.
Time elapsed: 2.49 s.
Completed DecisionTreeClassifier model's performance assessment.
Training GradientBoostingClassifier model...
Completed GradientBoostingClassifier model training.
Time elapsed: 8.11 s.
Completed GradientBoostingClassifier model's performance assessment.
Training AdaBoostClassifier model...
Completed AdaBoostClassifier model training.
Time elapsed: 1.97 s.
Completed AdaBoostClassifier model's 

### Result

ngram_range=(1, 2)
smooth_idf=True

In [127]:
df_perf_metrics_best

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.971963,0.917290,0.919858,0.922321,0.921088,1.17
1,MultinomialNB,0.983178,0.908411,0.870192,0.969643,0.917230,0.12
2,RandomForestClassifier,0.999844,0.903271,0.936842,0.874107,0.904388,11.09
3,DecisionTreeClassifier,1.000000,0.835981,0.830043,0.863393,0.846389,2.49
4,GradientBoostingClassifier,0.893769,0.854206,0.900794,0.810714,0.853383,8.11
5,AdaBoostClassifier,0.865576,0.847664,0.864220,0.841071,0.852489,1.97
6,LinearSVC,1.000000,0.937383,0.932456,0.949107,0.940708,0.05


## Export Overview to latex

In [128]:
result = pd.concat([df_perf_metrics_default[["Model","f1_score"]], df_perf_metrics_best[["f1_score"]]], axis=1)

In [129]:
result

,Model,f1_score,f1_score
0,LogisticRegression,0.923354,0.921088
1,MultinomialNB,0.915517,0.917230
2,RandomForestClassifier,0.906435,0.904388
3,DecisionTreeClassifier,0.853873,0.846389
4,GradientBoostingClassifier,0.860356,0.853383
5,AdaBoostClassifier,0.849932,0.852489
6,LinearSVC,0.935484,0.940708


In [130]:
print(result.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
                     Model &  f1\_score &  f1\_score \\
\midrule
        LogisticRegression &  0.923354 &  0.921088 \\
             MultinomialNB &  0.915517 &  0.917230 \\
    RandomForestClassifier &  0.906435 &  0.904388 \\
    DecisionTreeClassifier &  0.853873 &  0.846389 \\
GradientBoostingClassifier &  0.860356 &  0.853383 \\
        AdaBoostClassifier &  0.849932 &  0.852489 \\
                 LinearSVC &  0.935484 &  0.940708 \\
\bottomrule
\end{tabular}

